# Assignment 1 Task 1

In [2]:
import re
import string
import scipy
import pickle
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import *
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from astropy.table import Table, Column
import os
import glob
from nltk.corpus import stopwords
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from collections import defaultdict
from sklearn.feature_selection import VarianceThreshold

# Read files

In [5]:
df = pd.DataFrame(columns=['tweet','gender'])
for f in glob.glob('Assignment_4/twitter-gender-corpus/male/*.txt'):
    fil = open(f,encoding="utf8") 
    df = df.append(pd.DataFrame({'tweet': fil.read(), 'gender':'Male'}, index=[0]) , ignore_index=True)
    
for f in glob.glob('Assignment_4/twitter-gender-corpus/female/*.txt'):
    fil = open(f,encoding="utf8") 
    df = df.append(pd.DataFrame({'tweet': fil.read(), 'gender':'Feale'}, index=[0]) , ignore_index=True)

df

,tweet,gender
0,"@zulahni Oh, honey. Don&#39;t force us to stag...",Male
1,@lukeniuk Tack män! Kul som fan att träffas!\n...,Male
2,"Report from SIGIR 2011 (Beijing, China) - http...",Male
3,After quite some time working on different thi...,Male
4,Financial crisis hurting new job seekers in Ja...,Male
...,...,...
420,@pablomoreiras Exacto ;-) @eEAmerica\nLo q me ...,Feale
421,"&quot;Chance is like a picture, it&#39;d be ni...",Feale
422,How to Recognize a Hipster - Your server has a...,Feale
423,@rigbysdeli thanks so much! Info sent\nA moto ...,Feale


# Clean data

In [6]:

df.tweet = df.tweet.str.replace('[^\w\s]','',regex=True)
df

,tweet,gender
0,zulahni Oh honey Don39t force us to stage an i...,Male
1,lukeniuk Tack män Kul som fan att träffas\nTac...,Male
2,Report from SIGIR 2011 Beijing China httppost...,Male
3,After quite some time working on different thi...,Male
4,Financial crisis hurting new job seekers in Ja...,Male
...,...,...
420,pablomoreiras Exacto eEAmerica\nLo q me encan...,Feale
421,quotChance is like a picture it39d be nice if ...,Feale
422,How to Recognize a Hipster Your server has a ...,Feale
423,rigbysdeli thanks so much Info sent\nA moto fo...,Feale


# Label Encoding

In [7]:
ge = LabelEncoder()

df['gender'] = ge.fit_transform(df['gender'])
df

,tweet,gender
0,zulahni Oh honey Don39t force us to stage an i...,1
1,lukeniuk Tack män Kul som fan att träffas\nTac...,1
2,Report from SIGIR 2011 Beijing China httppost...,1
3,After quite some time working on different thi...,1
4,Financial crisis hurting new job seekers in Ja...,1
...,...,...
420,pablomoreiras Exacto eEAmerica\nLo q me encan...,0
421,quotChance is like a picture it39d be nice if ...,0
422,How to Recognize a Hipster Your server has a ...,0
423,rigbysdeli thanks so much Info sent\nA moto fo...,0


# Word uni-gram

In [17]:
v1 = CountVectorizer(max_features=500,ngram_range=(1, 1))
word_uni = v1.fit_transform(df.tweet).todense()
f = pd.DataFrame(word_uni, columns=v1.get_feature_names())
f

,10,100,12,20,2011,2012,2013,2014,30,about,...,yeah,year,years,yes,yet,york,you,you39re,your,youtube
0,3,1,7,0,0,0,2,2,5,35,...,17,10,9,13,8,1,206,33,56,4
1,9,4,4,3,0,2,1,0,1,42,...,47,6,1,13,3,0,153,15,19,0
2,4,1,2,4,29,25,16,4,1,27,...,0,9,3,3,1,3,44,5,32,4
3,3,0,1,0,5,0,1,0,3,36,...,4,3,2,1,3,0,35,4,12,1
4,6,5,1,5,8,13,5,2,3,21,...,0,20,12,0,2,1,60,1,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,2,0,0,0,0,0,2,4,0,4,...,0,4,0,0,0,0,18,0,6,0
421,3,1,4,2,3,2,1,0,1,19,...,0,5,5,9,0,35,111,14,36,1
422,4,0,0,0,0,2,0,0,0,4,...,2,2,2,6,0,0,32,0,6,0
423,7,2,0,2,1,0,6,1,0,87,...,2,5,1,7,6,0,133,11,53,7


# Evaluate models on Word uni-gram

In [18]:
x_train, x_test, y_train, y_test = train_test_split(word_uni, df.gender, test_size=0.30, random_state=30)

In [23]:
m1 = RandomForestClassifier()
m1.fit(x_train ,y_train)
p = m1.predict(x_test)
acc_rfc_uni = accuracy_score(y_test, p)
acc_rfc_uni


0.734375

In [24]:

m2 = LinearSVC()
m2.fit(x_train ,y_train)
p = m2.predict(x_test)
acc_lsvc_uni = accuracy_score(y_test, p)
acc_lsvc_uni

C:\Users\Abdullah\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.6796875

# Word bi-gram

In [25]:
v2 = CountVectorizer(max_features=500,ngram_range=(1, 2))
word_bi = v2.fit_transform(df.tweet).todense()
f = pd.DataFrame(word_bi, columns=v2.get_feature_names())
f

,10,100,12,20,2012,2013,2014,about,about the,actually,...,yet,york,you,you are,you can,you have,you know,you39re,your,youtube
0,3,1,7,0,0,2,2,35,4,11,...,8,1,206,9,13,12,11,33,56,4
1,9,4,4,3,2,1,0,42,2,11,...,3,0,153,3,2,1,7,15,19,0
2,4,1,2,4,25,16,4,27,2,0,...,1,3,44,3,3,2,0,5,32,4
3,3,0,1,0,0,1,0,36,5,3,...,3,0,35,1,2,3,0,4,12,1
4,6,5,1,5,13,5,2,21,4,1,...,2,1,60,8,4,4,1,1,15,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,2,0,0,0,0,2,4,4,0,0,...,0,0,18,0,0,2,0,0,6,0
421,3,1,4,2,2,1,0,19,0,3,...,0,35,111,3,5,2,2,14,36,1
422,4,0,0,0,2,0,0,4,0,0,...,0,0,32,0,2,2,2,0,6,0
423,7,2,0,2,0,6,1,87,20,10,...,6,0,133,5,7,5,6,11,53,7


# Evaluate models on Word bi-gram

In [26]:
x_train, x_test, y_train, y_test = train_test_split(word_bi, df.gender, test_size=0.30, random_state=30)

In [28]:
m1 = RandomForestClassifier()
m1.fit(x_train ,y_train)
p = m1.predict(x_test)
acc_rfc_bi = accuracy_score(y_test, p)
acc_rfc_bi


0.703125

In [29]:
m2 = LinearSVC()
m2.fit(x_train ,y_train)
p = m2.predict(x_test)
acc_lsvc_bi = accuracy_score(y_test, p)
acc_lsvc_bi

C:\Users\Abdullah\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.671875

# Char n-3

In [30]:
v3 = CountVectorizer(analyzer='char', ngram_range=(1,3), max_features=5000)
char3 = v3.fit_transform(df.tweet).todense()
f = pd.DataFrame(char3, columns=v3.get_feature_names())
f

,\n,\n1,\n2,\n3,\na,\na,\nad,\nal,\nam,\nan,...,ا,ال,ب,ر,ل,م,ن,و,ي,র
0,889,1,1,1,52,1,2,12,1,17,...,0,0,0,0,0,0,0,0,0,0
1,896,2,0,0,126,2,0,95,0,6,...,0,0,0,0,0,0,0,0,0,0
2,991,2,6,0,68,12,2,9,7,4,...,16,1,3,7,3,9,9,3,0,0
3,588,2,0,1,45,2,0,5,1,8,...,0,0,0,0,0,0,0,0,0,0
4,1001,13,9,7,124,12,56,12,2,11,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,156,0,0,0,18,0,0,12,0,4,...,0,0,0,0,0,0,0,0,0,0
421,796,1,1,0,56,11,0,6,5,13,...,0,0,0,0,0,0,0,0,0,0
422,444,2,0,0,18,2,0,2,4,4,...,0,0,0,0,0,0,0,0,0,0
423,977,0,2,1,73,5,1,10,3,27,...,0,0,0,0,0,0,0,0,0,0


# Evaluate models on Char n-3

In [31]:
x_train, x_test, y_train, y_test = train_test_split(char3, df.gender, test_size=0.30, random_state=30)

In [47]:
m1 = RandomForestClassifier()
m1.fit(x_train ,y_train)
p = m1.predict(x_test)
acc_rfc_char3 = accuracy_score(y_test, p)
acc_rfc_char3


0.7578125

In [48]:
m2 = LinearSVC()
m2.fit(x_train ,y_train)
p = m2.predict(x_test)
acc_lsvc_char3 = accuracy_score(y_test, p)
acc_lsvc_char3

C:\Users\Abdullah\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.6171875

# Char n-4

In [34]:
v4 = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=5000)
char4 = v4.fit_transform(df.tweet).todense()
f = pd.DataFrame(char4, columns=v4.get_feature_names())
f

,\n,\na,\na,\nal,\nan,\nb,\nbe,\nc,\nca,\nch,...,zin,zing,zn,znb,znbs,zo,zy,zz,í,ا
0,889,52,1,12,17,56,0,27,6,2,...,8,6,11,10,10,16,11,38,1,0
1,896,126,2,95,6,7,0,10,3,3,...,2,1,4,0,0,5,6,2,0,0
2,991,68,12,9,4,36,10,65,15,7,...,13,11,17,10,10,13,7,7,0,16
3,588,45,2,5,8,24,4,36,0,2,...,6,6,1,1,1,9,0,1,1,0
4,1001,124,12,12,11,31,4,67,14,12,...,6,3,5,3,3,5,1,10,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,156,18,0,12,4,8,2,10,6,2,...,2,2,2,2,2,12,0,2,26,0
421,796,56,11,6,13,31,10,38,13,8,...,26,19,6,5,5,8,7,8,0,0
422,444,18,2,2,4,10,6,18,6,8,...,10,10,10,6,6,4,2,6,0,0
423,977,73,5,10,27,41,10,33,7,2,...,42,40,9,5,5,6,4,13,4,0


# Evaluate models on Char n-4

In [35]:
x_train, x_test, y_train, y_test = train_test_split(char4, df.gender, test_size=0.30, random_state=30)

In [49]:

m1 = RandomForestClassifier()
m1.fit(x_train ,y_train)
p = m1.predict(x_test)
acc_rfc_char4 = accuracy_score(y_test, p)
acc_rfc_char4

0.765625

In [50]:
m2 = LinearSVC()
m2.fit(x_train ,y_train)
p = m2.predict(x_test)
acc_lsvc_char4 = accuracy_score(y_test, p)
acc_lsvc_char4

C:\Users\Abdullah\anaconda3\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


0.6328125

# Best model

In [54]:
print('Accuracy of RandomForestClassifier on Word unigram:\t\t', acc_rfc_uni)
print('Accuracy of LinearSVC on CWord unigram:\t\t\t\t', acc_lsvc_uni)

print('Accuracy of RandomForestClassifier on word bigram:\t\t', acc_lsvc_bi)
print('Accuracy of LinearSVC on Word bigram:\t\t\t\t', acc_lsvc_bi)

print('Accuracy of RandomForestClassifier on Char n-3:\t\t\t', acc_rfc_char3)
print('Accuracy of LinearSVC on Char n-3:\t\t\t\t', acc_lsvc_char3)

print('Accuracy of RandomForestClassifier on Char n-4:\t\t\t', acc_rfc_char4)
print('Accuracy of LinearSVC on Char n-4:\t\t\t\t', acc_lsvc_char4)

Accuracy of RandomForestClassifier on Word unigram:		 0.7421875
Accuracy of LinearSVC on CWord unigram:				 0.6796875
Accuracy of RandomForestClassifier on word bigram:		 0.671875
Accuracy of LinearSVC on Word bigram:				 0.671875
Accuracy of RandomForestClassifier on Char n-3:			 0.7578125
Accuracy of LinearSVC on Char n-3:				 0.6171875
Accuracy of RandomForestClassifier on Char n-4:			 0.765625
Accuracy of LinearSVC on Char n-4:				 0.6328125


In [58]:
print('\nBest Model is:\n')
print('Accuracy of RandomForestClassifier on Char n-4:\t\t\t', acc_rfc_char4)


Best Model is:

Accuracy of RandomForestClassifier on Char n-4:			 0.765625


# Train Best Model

In [59]:
v4 = CountVectorizer(analyzer='char', ngram_range=(1,4), max_features=5000)
char4 = v4.fit_transform(df.tweet).todense()
f = pd.DataFrame(char4, columns=v4.get_feature_names())
f

,\n,\na,\na,\nal,\nan,\nb,\nbe,\nc,\nca,\nch,...,zin,zing,zn,znb,znbs,zo,zy,zz,í,ا
0,889,52,1,12,17,56,0,27,6,2,...,8,6,11,10,10,16,11,38,1,0
1,896,126,2,95,6,7,0,10,3,3,...,2,1,4,0,0,5,6,2,0,0
2,991,68,12,9,4,36,10,65,15,7,...,13,11,17,10,10,13,7,7,0,16
3,588,45,2,5,8,24,4,36,0,2,...,6,6,1,1,1,9,0,1,1,0
4,1001,124,12,12,11,31,4,67,14,12,...,6,3,5,3,3,5,1,10,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
420,156,18,0,12,4,8,2,10,6,2,...,2,2,2,2,2,12,0,2,26,0
421,796,56,11,6,13,31,10,38,13,8,...,26,19,6,5,5,8,7,8,0,0
422,444,18,2,2,4,10,6,18,6,8,...,10,10,10,6,6,4,2,6,0,0
423,977,73,5,10,27,41,10,33,7,2,...,42,40,9,5,5,6,4,13,4,0


In [60]:
m1.fit(char4 ,df.gender)

RandomForestClassifier()